# Image Stacking
---
- Author: Diego Inácio
- GitHub: [github.com/diegoinacio](https://github.com/diegoinacio)
- Notebook: [stacking.ipynb](https://github.com/diegoinacio/computer-vision-notebooks/blob/master/Computer-Vision-Fundamentals/stacking.ipynb)
---
Implementation of solution to image stacking and statistical blending.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import imageio
import numpy as np
import scipy.stats as stats
import glob

from _utils import *

## Sample
---

In [ ]:
sample = imageio.imread('../_data/SEQ01-32bits/sample.0001.exr')

### Sample analysis
---

In [ ]:
print('sample 0001')
summary(sample)

In [ ]:
histogram(sample, interval=[0, 1])

### Colorspace transformation
---
Transform from *linear* to *sRGB* usign gamma correction $\large \gamma = 2.2$.

In [ ]:
sample_sRGB = sample**(1/2.2)

In [ ]:
histogram(sample_sRGB, interval=[0, 1])

## Stacking
---

In [ ]:
def stackRead(pathname):
    '''
    pathname defined by "glob" pattern.
    i.e.: "directory/sequence_folder/image_*.jpg"
    '''
    # List of image in pathname folder
    SEQ_IMG = glob.glob(pathname)
    n = len(SEQ_IMG)
    # sample for stack definition
    sample = imageio.imread(SEQ_IMG[0])
    # x and y are the dimensions
    # c is the number of channels
    y, x, c = sample.shape
    # define stack
    stack = np.zeros((n, y, x, c), dtype=sample.dtype)
    # image stacking
    for FILE in SEQ_IMG:
        index = SEQ_IMG.index(FILE)
        stack[index] = imageio.imread(FILE)
    # output
    return stack

In [ ]:
stack = stackRead('../_data/SEQ01-32bits/sample.*.exr')

In [ ]:
panel(stack**(1/2.2), (3, 1),
      interval=[0, 1],
      dims=(1200, 400),
      texts=['{:04}'.format(i + 1) for i in range(10)])

## Blend operations
---

In [ ]:
def blendStack(stack, modo='median', axis=0):
    if modo == 'sum':
        blend = np.sum(stack, axis)
        
    if modo == 'arithmetic mean':
        blend = np.mean(stack, axis)
    
    if modo == 'geometric mean':
        blend = stats.gmean(stack, axis)
    
    if modo == 'harmonic mean':
        blend = stats.hmean(stack, axis)
    
    if modo == 'median':
        blend = np.median(stack, axis)
    
    if modo == 'minimum':
        blend = np.amin(stack, axis)

    if modo == 'maximum':
        blend = np.amax(stack, axis)

    if modo == 'curtosis':
        blend = stats.kurtosis(stack, axis)

    if modo == 'variance':
        blend = np.var(stack, axis)

    if modo == 'standard deviation':
        blend = np.std(stack, axis)

    return blend.astype(stack.dtype)

### Analysis
---

In [ ]:
median = blendStack(stack)
summary(median)

In [ ]:
histogram(median**(1/2.2), interval=[0, 1])

In [ ]:
sample_blend = np.array([stack[0]**(1/2.2), median**(1/2.2)])
panel(sample_blend, (2, 1),
      interval=[0, 1],
      texts=['sample 0001', 'median'])

## Examples
---

### Sum
---

In [ ]:
blend = blendStack(stack**(1/2.2), modo='sum')
summary(blend)

In [ ]:
histogram(blend, interval=[5, 38])

In [ ]:
sample_blend = np.array([stack[0]**(1/2.2), (blend - 5)/(38 - 5)])
panel(sample_blend, (2, 1),
      interval=[0, 1],
      texts=['sample 0001', 'sum'])

### Means
---

In [ ]:
mean_a = blendStack(stack**(1/2.2), modo='arithmetic mean')
mean_g = blendStack(stack**(1/2.2), modo='geometric mean')
mean_h = blendStack(stack**(1/2.2), modo='harmonic mean')

In [ ]:
sample_blend = np.array([mean_a, mean_g, mean_h])
panel(sample_blend, (3, 1),
      dims=(1200, 400),
      interval=[0, 1],
      texts=['arithmetic mean', 'geometric mean', 'harmonic mean'])

### Extremes
---

In [ ]:
minimum = blendStack(stack**(1/2.2), modo='minimum')
maximum = blendStack(stack**(1/2.2), modo='maximum')

In [ ]:
sample_blend = np.array([minimum, maximum])
panel(sample_blend, (2, 1),
      interval=[0, 1],
      texts=['minimum', 'maximum'])